In [2]:
import pandas as pd
import numpy as np
import sys
from knn_imputer.knnimpute import KNNImputer
from categorical_imputer import CategoricalImputer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 25)
np.set_printoptions(suppress=True, threshold=sys.maxsize)

In [3]:
telcomdata = pd.read_csv("customer_churn.tsv")
telcomdata['TotalCharges']=pd.to_numeric(telcomdata['TotalCharges'],errors="coerce")
telcomdata['SeniorCitizen']=telcomdata['SeniorCitizen'].astype('category')
    
for i in telcomdata.columns:
    if telcomdata[i].dtypes == "O":
        telcomdata[i] = telcomdata[i].astype('category')
    else:
        pass

In [4]:
telcomdata.dtypes

customerID          category
gender              category
SeniorCitizen       category
Partner             category
Dependents          category
tenure               float64
PhoneService        category
MultipleLines       category
InternetService     category
OnlineSecurity      category
OnlineBackup        category
DeviceProtection    category
TechSupport         category
StreamingTV         category
StreamingMovies     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
MonthlyCharges       float64
TotalCharges         float64
Churn               category
dtype: object

In [5]:
X = telcomdata._get_numeric_data()
imputer = KNNImputer(n_neighbors=2, weights="uniform")
np.set_printoptions(suppress=True, threshold=sys.maxsize)
result = imputer.fit_transform(X)
telcomdata[list(telcomdata._get_numeric_data().columns)] = result


/home/dell/Desktop/ML_based_Churn_Prediction/knn_imputer/knnimpute.py:222: UserWarning: There are rows with more than 50.0% missing values. These rows are not included as donor neighbors.
  .format(self.row_max_missing * 100))
/home/dell/Desktop/ML_based_Churn_Prediction/knn_imputer/knnimpute.py:280: UserWarning: There are rows with more than 50.0% missing values. The missing features in these rows are imputed with column means.
  .format(self.row_max_missing * 100))


In [6]:
telcomdata.dtypes

customerID          category
gender              category
SeniorCitizen       category
Partner             category
Dependents          category
tenure               float64
PhoneService        category
MultipleLines       category
InternetService     category
OnlineSecurity      category
OnlineBackup        category
DeviceProtection    category
TechSupport         category
StreamingTV         category
StreamingMovies     category
Contract            category
PaperlessBilling    category
PaymentMethod       category
MonthlyCharges       float64
TotalCharges         float64
Churn               category
dtype: object

In [7]:
catagorical = ['category']
imputer_cat = CategoricalImputer()

for i in list(telcomdata[list(telcomdata.select_dtypes(include=catagorical).columns)].columns):
    column = telcomdata[i]
    column.to_numpy()
    try:
        result = imputer_cat.fit_transform(column)
        telcomdata[i] = result
        telcomdata[i] = telcomdata[i].astype('category')
    except:
        pass

In [8]:
from sklearn import preprocessing

number = preprocessing.LabelEncoder()
for i in list(telcomdata.columns):
    if not telcomdata[i].dtype == float:
        telcomdata[i] = number.fit_transform(telcomdata[i])


In [9]:
y=telcomdata["Churn"]
x=telcomdata.drop('Churn',axis=1)
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [10]:
telcomdata.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [11]:
rf = RandomForestClassifier(random_state=0,n_estimators=1000,criterion="gini")
rf.fit(xTrain, yTrain) 

train_acc_rfc = round((rf.score(xTrain, yTrain))*100,1)
test_acc_rfc = round((rf.score(xTest, yTest))*100,1)

print("Training accuracy: %s %%" % train_acc_rfc)
print('Test accuracy: %s %%' % test_acc_rfc)

Training accuracy: 100.0 %
Test accuracy: 78.4 %


In [12]:
rfc = RandomForestClassifier(random_state=0,n_estimators=1000)
rfc.fit(xTrain, yTrain)
importances = rfc.feature_importances_

f_importances = pd.Series(importances, telcomdata.columns[:-1])
f_importances.sort_values(ascending=False, inplace=True)
f_importances

TotalCharges        0.153520
MonthlyCharges      0.145163
tenure              0.142106
customerID          0.127917
Contract            0.078003
PaymentMethod       0.043936
OnlineSecurity      0.041604
TechSupport         0.038778
OnlineBackup        0.026932
InternetService     0.026759
gender              0.023096
PaperlessBilling    0.021620
DeviceProtection    0.020813
MultipleLines       0.020489
Partner             0.019010
SeniorCitizen       0.017891
Dependents          0.016472
StreamingTV         0.015704
StreamingMovies     0.015374
PhoneService        0.004812
dtype: float64